# Interacting with the Blackboard

In [13]:
import urllib, requests, json, time
bb_url = 'https://translator.ncats.io/blackboard/api/kg'
#bb_url = 'http://localhost:9000/blackboard/api/kg'

In [14]:
#query = {"type": "query", "name": "A simple blackboard example","term": "asthma"}
query = {"type":"drug", "uri":"https://pharos.ncats.io/idg/api/v1/ligands(31242)", 
         "name":"imatinib", "description": "This is an example query seeded with gleevec"}
json.dumps(query)
req = requests.post(bb_url, json=query)
req.status_code

200

Having posted the query, we then retrieve the resultant knowledge graph (KG) is JSON form. The blackboard can contain multiple knowledge graphs (one for each query submitted to it). So we identify the KG with the highest id, which should correspond to the query we just submitted

In [15]:
kg = requests.get(bb_url).json()
latest_kg_id = max([x['id'] for x in kg])
kg_url = bb_url+"/"+str(latest_kg_id)
print('Currently have %d KGs. Will use KG ID=%d, %s' % (len(kg), latest_kg_id, kg_url))

Currently have 2 KGs. Will use KG ID=33, https://translator.ncats.io/blackboard/api/kg/33


We then update the KG using the Pharos KS (note this needs to be a `PUT` request)

In [16]:
req = requests.put(kg_url+"/ks.pharos")
req.status_code
time.sleep(10)

Now we retrieve the current state of the KG. By default we get a condensed representation of the KG. To get the complete JSON representation we append `?view=full`


In [17]:
req = requests.get(kg_url+"?view=full")
req.json()

{'edgeCount': 6,
 'edges': [{'directed': False,
   'id': 173,
   'source': 34,
   'target': 35,
   'type': 'inhibitor'},
  {'directed': False,
   'id': 174,
   'source': 34,
   'target': 36,
   'type': 'inhibitor'},
  {'directed': False,
   'id': 175,
   'source': 34,
   'target': 37,
   'type': 'inhibitor'}],
 'id': 33,
 'name': 'imatinib',
 'nodeCount': 4,
 'nodes': [{'degree': 3,
   'description': 'This is an example query seeded with gleevec',
   'id': 34,
   'inDegree': 0,
   'kgraph': 33,
   'name': 'imatinib',
   'outDegree': 3,
   'tags': ['KQuery', 'drug', 'KG:33'],
   'type': 'drug',
   'uri': 'https://pharos.ncats.io/idg/api/v1/ligands(31242)'},
  {'degree': 1,
   'id': 35,
   'inDegree': 1,
   'kgraph': 33,
   'name': 'Tyrosine-protein kinase ABL1',
   'outDegree': 0,
   'synonyms': ['4J9D',
    '4J9F',
    '5DC0',
    '5DC4',
    '5DC9',
    '1AB2',
    '1ABL',
    '1AWO',
    '1BBZ',
    '1JU5',
    '1OPL',
    '1ZZP',
    '2ABL',
    '2E2B',
    '2F4J',
    '2FO0',
    '

As noted in the [documentation](https://spotlite.nih.gov/ncats/blackboard) we can rerun the Pharos KS to add more nodes and edges. The first call, above, added nodes that were related to the query node (and the associated edges). When we call it again, it will add nodes that are related to the nodes added in the previous run (i.e., 2nd neighbors of the query node)

In [18]:
req = requests.put(kg_url+"/ks.pharos")
time.sleep(30)

In [19]:
## Dump out the KG JSON for inspection
req = requests.get(kg_url+"?view=full")
ofile = open('kg.json', 'w') ## for reference
ofile.write(req.text)
ofile.close()

Currently the knowledge graph is stored in a [Neo4J](https://neo4j.com/) instance. We can interact with this assuming you have a Neo4J client running and pointed to `blackboard.db` that was created by the blackboard API.

Alternatively, we can parse the JSON representation of the knowledge graph into an `igraph` object and then visualize it using the Cytoscape [extension](https://github.com/cytoscape/jupyter-cytoscape) for Jupyter notebooks. (This is most easily installed if you're using the Anaconda distribution of Python). 

First we make a function to go from the Knowlegde Graph JSON to an `igraph` object

In [28]:
from igraph import *

def kg2ig(kg):
    if kg['type'] != 'kgraph':
        raise Exception("Must provide a JSON kgraph")

    g = Graph(directed=False)

    nodes = kg['nodes']
    for node in nodes:
        d = {}
        for key in node.keys():
            if key in ['inDegree','outDegree','degree', 'id']: continue
            #key = key.encode("ascii")
            d[key] = node[key]
        g.add_vertex(**d)

    edges = kg['edges']
    for edge in edges:
        s = list(filter(lambda x: x['id'] == edge['source'], nodes))
        t = list(filter(lambda x: x['id'] == edge['target'], nodes))
        if len(s) == 1 and len(t) == 1:
            g.add_edge(s[0]['name'],t[0]['name'], type=edge['type'])
        
    return g

We then use this to get the object and then display it via the Cytoscape extension

In [29]:
req = requests.get(kg_url+"?view=full")
g = kg2ig(json.loads(req.text))
print(g.summary())

IGRAPH UN-T 39 70 -- 
+ attr: description (v), kgraph (v), name (v), synonyms (v), tags (v), type (v), uri (v), type (e)


In [30]:
from cyjs import *
cy = cyjs()
display(cy)

In [31]:
cy.deleteGraph()
cy.addGraph(g)
kkLayout = g.layout("kk")
cy.setPosition(kkLayout)
cy.fit(10)

In [32]:
cy.loadStyleFile('bb-style.js')

Given the current network, we can push it to [Ndex](http://www.ndexbio.org/). You should have created an account, which you'll need to push the KG into Ndex.

In [33]:
from ndex.networkn import NdexGraph
import ndex.client as nc
import uuid
ng = NdexGraph()
ng.set_name(kg_url)
for n in g.vs:
    ng.add_new_node(n['name'], n.attributes())
for idx, e in enumerate(g.es):
    eid = ng.add_edge_between(e.source+1, e.target+1)
    ng.set_edge_attribute(eid, 'type', e['type'])

client = nc.Ndex("http://dev.ndexbio.org", 'foobar123', 'hello123')
uri = client.save_new_network(ng.to_cx())
uuid = uri.rpartition('/')[-1]
##client.make_network_public(uuid)
print("KG is on Ndex at %s" % (uri))

AttributeError: 'IndexError' object has no attribute 'message'

Once the KG has been pushed to the Ndex server, you can search for it using the KG ID (or URL)